# Install Dependencies

In [60]:
!pip install tensorflow==2.3.0
!pip install gym==0.25.2
!pip install keras
!pip install keras-rl2==1.0.5
!pip install rl-agents==0.1.1

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0)
ERROR: No matching distribution found for tensorflow==2.3.0
You should consider upgrading via the 'k:\python\python39\python.exe -m pip install --upgrade pip' command.


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gym: filename=gym-0.25.2-py3-none-any.whl size=852296 sha256=5d6d78920d7ae57f4ce282dee004bb131e986b25c79a26a7e4791fa2b213db9f
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\05\4d\6c\d0ef0db36695ce032fe20099e3149d8db85cf36656176ff745
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

rl-agents 0.1.1 requires gym<0.18.0,>=0.17.2, but you'll have gym 0.25.2 which is incompatible.
You should consider upgrading via the 'k:\python\python39\python.exe -m pip install --upgrade pip' command.



    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3


You should consider upgrading via the 'k:\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'k:\python\python39\python.exe -m pip install --upgrade pip' command.


Processing c:\users\user\appdata\local\pip\cache\wheels\5d\06\a4\57a926b2e87a5e1f21551577750549206dde639e19a5ac72d5\gym-0.17.3-py3-none-any.whl
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


You should consider upgrading via the 'k:\python\python39\python.exe -m pip install --upgrade pip' command.


# Test Random Environment with OpenAI Gym

In [61]:
import gym
import random

In [62]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [72]:
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v0>>>>>

In [64]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        _, n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:181
Episode:2 Score:183
Episode:3 Score:183
Episode:4 Score:162
Episode:5 Score:182
Episode:6 Score:173
Episode:7 Score:177
Episode:8 Score:182
Episode:9 Score:186
Episode:10 Score:169


# Create a Deep Learning Model with Keras

In [65]:
import numpy as np
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.python.keras.optimizers import adam_v2

In [66]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [67]:
model = build_model(states, actions)

In [68]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# Build Agent with Keras-RL

In [69]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [70]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [71]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_1_input to have shape (1, 4) but got array with shape (1, 2)

# Reloading Agent from Memory